# Data Loading and fature extraction
below you will see that we load data with three techniques each of them will work with one of the three models

Loading Data from Kaggle

In [19]:
!pip install -q kaggle 
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [20]:
!kaggle datasets download -d kameo4189/aflw2000-300wlp

100% 3.65G/3.65G [00:35<00:00, 110MB/s]
100% 3.65G/3.65G [00:35<00:00, 111MB/s]


In [3]:
%%capture
!pip install mediapipe

In [4]:
import numpy as np
import os,cv2,math,glob,random
import scipy.io as sio
from math import cos, sin
from pathlib import Path
import pandas as pd
import mediapipe
import warnings
warnings.filterwarnings('ignore')
from google.colab.patches import cv2_imshow
import glob

In [ ]:
%%capture
!unzip /content/aflw2000-300wlp.zip

In [24]:
def load_data(load_type='whole'):
  data = dict()
  faceModule = mediapipe.solutions.face_mesh
  with faceModule.FaceMesh(static_image_mode=True) as faces:
    for img in glob.glob("/content/AFLW2000-3D/AFLW2000/*.jpg"):
        # loading the image
        image = cv2.imread(img)
        # processing the face to extract the landmark points (468 point) for each x,y,z
        results = faces.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if results.multi_face_landmarks != None: 
          # looping over the faces in the image
          face = results.multi_face_landmarks[0]
          for i,landmark in enumerate(face.landmark):
            # nose is the index 1 in landmarks
              x = landmark.x
              y = landmark.y
              if load_type == 'whole':
                  
                  data.setdefault(str(i)+'_x',[])
                  data[str(i)+'_x'].append(x)
                  data.setdefault(str(i)+'_y',[])
                  data[str(i)+'_y'].append(y)
              elif load_type == 'avg':
                  data.setdefault(str(i),[])
                  data[str(i)].append((x*0.7+y*0.3))
              elif load_type == 'angle':
                  dx = x - face.landmark[1].x
                  dy = y - face.landmark[1].y
                  theta = math.atan2(dy, np.sqrt(dx*dx))
                  data.setdefault(str(i),[])
                  data[str(i)].append(theta)
              else: return None
          mat_file = sio.loadmat('mat'.join(img.split('jpg')))
          # extracting the labels 3 angels
          pose_para = mat_file["Pose_Para"][0][:3]
          pitch = pose_para[0]
          yaw = pose_para[1]
          roll = pose_para[2]
          data.setdefault('pitch',[])
          data['pitch'].append(pitch)
          data.setdefault('yaw',[])
          data['yaw'].append(yaw)   
          data.setdefault('roll',[])
          data['roll'].append(roll) 
  return data   

Load ALL X,y Data more approbriate for pitch model

In [27]:
data_df = pd.DataFrame(load_data('whole'))
data_df.to_csv('pose_estimate.csv',index=False)

Extract face feature and instead of recording each x, y coordinate of the feature i will record average of them
by understanding the problem and after alot of trials this average is the best for modeling yaw

In [28]:
data_compressed_df = pd.DataFrame(load_data('avg'))
data_compressed_df.to_csv('pose_estimate_avgComposed.csv',index=False)      

Extract face feature and instead of recording each x, y coordinate of the feature i will record the ange between each point and the noise.

this angle is the best calculation for roll

In [29]:
data_compArctan_df = pd.DataFrame(load_data('angle'))
data_compArctan_df.to_csv('pose_estimate_arctanComposed.csv',index=False)    